In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import seq2seq

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 32
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556917
['\ufeffThe Project Gutenberg EBook of ', 'The Project Gutenberg EBook of C', 'he Project Gutenberg EBook of Ch', 'e Project Gutenberg EBook of Che', ' Project Gutenberg EBook of Ches']



torch.Size([556917, 32])
torch.Size([556917, 16])
torch.Size([556917, 17])


# fit

In [8]:
importlib.reload(seq2seq)  

net = seq2seq.LSTM(char2i, i2char)
net.to(device)
performance = net.fit(source_1, target_1, save_path = "checkpoints/seq2seq_lstm.pt", progress_bar = 0)
net.save_architecture("architectures/seq2seq_lstm.architecture")

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Encoder embedding dimension: 32
Decoder embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   1.2396 |     35.408 |      1.4
    2 |   0.8837 |     27.066 |      2.8
    3 |   0.8020 |     24.921 |      4.2
    4 |   0.7540 |     23.642 |      5.7
    5 |   0.7205 |     22.732 |      7.1


In [9]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

,epoch,train_loss,train_error_rate,minutes,learning_rate,weight_decay,model,encoder_embedding_dimension,decoder_embedding_dimension,encoder_hidden_units,encoder_layers,decoder_hidden_units,decoder_layers,dropout,parameters
0,1,1.239611,35.408093,1.413818,0.001,0,Seq2Seq LSTM,32,32,128,2,128,2,0.0,579487
1,2,0.883674,27.066017,2.830199,0.001,0,Seq2Seq LSTM,32,32,128,2,128,2,0.0,579487
2,3,0.802045,24.920915,4.249755,0.001,0,Seq2Seq LSTM,32,32,128,2,128,2,0.0,579487
3,4,0.753978,23.641730,5.671105,0.001,0,Seq2Seq LSTM,32,32,128,2,128,2,0.0,579487
4,5,0.720463,22.731978,7.087438,0.001,0,Seq2Seq LSTM,32,32,128,2,128,2,0.0,579487


In [10]:
# the input for testing

net.tensor2text(source_1[:5])

['\ufeffThe Project Gut',
 'The Project Gute',
 'he Project Guten',
 'e Project Gutenb',
 ' Project Gutenbe']

# predict

In [11]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)

idx, log_probabilities = net.predict(source_1[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Encoder embedding dimension: 32
Decoder embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487



['<START>enberg-tm electr',
 '<START>nberg-tm electro',
 '<START>berg-tm electron',
 '<START>erg-tm electroni',
 '<START>rg-tm electronic']

# greedy_search

In [12]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(source_1[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Encoder embedding dimension: 32
Decoder embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

tensor([-2.3400, -2.2834, -2.2556, -2.9405, -2.2111], device='cuda:0')


['<START>enberg-tm electronic',
 '<START>nberg-tm electronic ',
 '<START>berg-tm electronic w',
 '<START>erg-tm electronic wo',
 '<START>rg-tm electronic wor']

# sample

In [15]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(source_1[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Encoder embedding dimension: 32
Decoder embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

tensor([-19.2702, -21.5866, -31.3541, -11.2698,  -9.0979], device='cuda:0')


['<START>enberg-tm threlegree',
 '<START>ptony his distrodect',
 '<START>berg (108 is enoorat',
 '<START>erg Literary Johused',
 '<START>rg-tm\nelectronic the']

# beam_search

In [21]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(source_1[:5], progress_bar = 0)

print(log_probabilities)
[net.tensor2text(t) for t in indexes]

Model: Seq2Seq LSTM
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Encoder embedding dimension: 32
Decoder embedding dimension: 32
Encoder hidden units: 128
Encoder layers: 2
Decoder hidden units: 128
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 579,487

tensor([[-2.3400, -2.6164, -3.6843, -5.1370, -5.5572],
        [-2.2834, -3.2239, -3.3584, -3.8615, -4.9356],
        [-2.2556, -3.1783, -3.9974, -4.1785, -4.9287],
        [-2.9405, -3.6931, -4.0829, -4.4183, -5.1069],
        [-2.2111, -3.1253, -3.6750, -4.4433, -4.7890]], device='cuda:0')


[['<START>enberg-tm electronic',
  '<START>enberg Literary Arch',
  '<START>enberg-tm\nelectronic',
  '<START>enberg-tm electrowin',
  '<START>enberg-tm electrogra'],
 ['<START>nberg-tm electronic ',
  '<START>nberg Literary Archi',
  '<START>nberg-tm\nelectronic ',
  '<START>nberg-tm electronic\n',
  '<START>nberg-tm\nelectronic\n'],
 ['<START>berg-tm electronic w',
  '<START>berg-tm\nelectronic w',
  '<START>berg-tm electronic\nw',
  '<START>berg Literary Archiv',
  '<START>berg-tm\nelectronic\nw'],
 ['<START>erg-tm electronic wo',
  '<START>erg Literary Archive',
  '<START>erg-tm\nelectronic wo',
  '<START>erg-tm electronic\nwo',
  '<START>erg-tm electronic an'],
 ['<START>rg-tm electronic wor',
  '<START>rg-tm\nelectronic wor',
  '<START>rg-tm electronic\nwor',
  '<START>rg Literary Archive ',
  '<START>rg-tm electronic and']]